In [ ]:
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from io import StringIO 
from sklearn.metrics import mean_squared_error,roc_auc_score,precision_score

from cancellation_model import CancellationModel, load_training_data, load_prediction_data


In [ ]:
train = load_training_data()
test = load_prediction_data()

In [ ]:
params={}
params['learning_rate']=0.03
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='binary' #Binary target feature
params['metric']='binary_logloss' #metric for binary classification
params['max_depth']=10

model = CancellationModel({"train_params" : params, "epocs" : 100})
model.train(train)
X_test['pred'], X_test['cancelled'] = model.predict(test)

In [ ]:
roc_auc_score(X_test.cancelled, X_test.pred)
roc_auc_score(X_test.cancelled, X_test.pred)

In [ ]:
X_test.pred.sum()/X_test.cancelled.sum()